In [ ]:
from azmeta.access import resource_graph, list_subscription_ids
from azmeta.notebook.context import AzureCliContext
from azmeta.notebook.utils import uncommon_substrings
import pandas as pd
import itertools

# Parameters

In [ ]:
resource_filter = None

# Log Analytics Workspace Selection

In [ ]:
context = AzureCliContext()
all_subscription_ids = list_subscription_ids(context.subscriptions)
workspaces = resource_graph.query_dataframe(all_subscription_ids, f"""
Resources 
| where type == 'microsoft.operationalinsights/workspaces'
| where {resource_filter if resource_filter else "1 == 1"}
| project subscriptionId, resourceGroup, name
""")
workspaces = workspaces.assign(alias=uncommon_substrings(workspaces.name.to_list()))

In [ ]:
workspaces

# Pull Data Source Configuration

In [ ]:
from azure.common import credentials
from azure.mgmt.loganalytics import LogAnalyticsManagementClient
creds, _ = credentials.get_azure_cli_credentials()
filter = "kind eq 'windowsPerformanceCounter'"
queries = \
    (((w.alias, ds) for ds in LogAnalyticsManagementClient(creds, w.subscriptionId).data_sources.list_by_workspace(w.resourceGroup, w.name, filter)) 
    for w in workspaces.itertuples())

In [ ]:
data = [(ds[0], f"{ds[1].properties['objectName']} | {ds[1].properties['counterName']}", ds[1].properties['intervalSeconds']) for ds in itertools.chain.from_iterable(queries)]

# Report

In [ ]:
df = pd.DataFrame(data, columns=["alias", "counter", "interval"])
pt = pd.pivot_table(df, index="counter", columns="alias", values="interval")

In [ ]:
pt.style.background_gradient(vmin=10, vmax=600).highlight_null('black').format("{:g}", na_rep='NA')